# **DATA ANALYSIS - "PICK_DATA"**

## Packages

### Installing packages

In [1]:
# !pip3 install numpy
# !pip3 install --upgrade pip
# !pip3 install pandas
# !pip3 install matplotlib
# ! pip3 install seaborn

### Importing packages

In [2]:
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output

## Functions

Function that returns an overview of the **pick_data** based on all columns

In [3]:
# function that returns an overview of the pick_data based on all columns
def general_overview(pick_data):

    #get unique values for each variable
    unique_number_of_products = len(set(pick_data.product_id.values))
    unique_number_of_categories = len(set(pick_data.category.values))
    unique_number_of_source_order_ids = len(set(pick_data.source_order_id.values))
    unique_number_of_order_numbers = len(set(pick_data.order_number.values))
    unique_number_of_positions_in_order = len(set(pick_data.position_in_order.values))
    unique_number_of_quantity_unit = len(set(pick_data.quantity_unit.values))

    # print the results for a better understanding
    print("The length of the data is: ", len(pick_data))
    print("-----")
    print("-----")
    print("product_id")
    print("The number of unique product_id is: ",  unique_number_of_products)
    print("-----")
    print("category")
    print("The number of unique category is: ", unique_number_of_categories)
    print("-----")
    print("source_order_id")
    print("The number of unique source_order_id is: ", unique_number_of_source_order_ids)
    print("-----")
    print("order_number")
    print("The number of unique order_number is: ", unique_number_of_order_numbers)
    print("-----")
    print("position_in_order")
    print("The number of unique position_in_order is: ", unique_number_of_positions_in_order)
    print("-----")
    print("pick_volume")
    print("The min value of pick_volume is: ", pick_data.pick_volume.min())
    print("The max value of pick_volume is: ", pick_data.pick_volume.max())
    print("-----")
    print("quantity_unit")
    print("The number of unique quantity_unit is: ", unique_number_of_quantity_unit)
    print("-----")
    print("date_time")
    print("The first date_time of the picks_data is: ", pick_data.date_time.min())
    print("The last date_time of the picks_data is: ", pick_data.date_time.max())

Function that checks duplicate rows showing their duplicated partners based on a column selection

In [4]:
def check_duplicates_row_columns(pick_data, index_to_check, columns_to_evaluate):

    index_row_pick_data = pick_data.loc[index_to_check, columns_to_evaluate]

    matching_rows = pick_data[pick_data[columns_to_evaluate].eq(index_row_pick_data).all(axis=1)]
    
    return(matching_rows)

Function that checks if integer list is sequential

In [5]:
def is_sequential(int_list):
    # Sort the list first (if not already sorted)
    sorted_list = sorted(int_list)

    # Check if each consecutive number differs by 1
    for i in range(len(sorted_list) - 1):
        if sorted_list[i + 1] != sorted_list[i] + 1:
            return False

    return True

Function that counts the number of times a list is not sequential

In [6]:
def count_non_sequential(int_list):
    # Sort the list first (if not already sorted)
    sorted_list = sorted(int_list)

    # Initialize a counter for non-sequential occurrences
    non_sequential_count = 0

    # Check each consecutive number
    for i in range(len(sorted_list) - 1):
        if sorted_list[i + 1] != sorted_list[i] + 1:
            non_sequential_count += 1

    return non_sequential_count

Function that identifies the **outliers** in **product_id** taking parameters **Q1**, **Q3**

In [7]:
def product_id_volume_outliers(Q1, Q3, df, product_id):
    
    pick_data_volume_evaluation = df.copy()
    pick_data_volume_evaluation = pick_data_volume_evaluation[pick_data_volume_evaluation["product_id"] == product_id]
    pick_data_volume_evaluation = pick_data_volume_evaluation.reset_index(drop=True)

    Q1 = pick_data_volume_evaluation['pick_volume'].quantile(Q1)
    Q3 = pick_data_volume_evaluation['pick_volume'].quantile(Q3)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define bounds for outliers
    # lower_bound = Q1 - 1.5 * IQR
    lower_bound = 0
    upper_bound = Q3 + 1.5 * IQR

    print(lower_bound, upper_bound)

    # Identify outliers
    outliers = pick_data_volume_evaluation[(pick_data_volume_evaluation["pick_volume"] < lower_bound) | (pick_data_volume_evaluation["pick_volume"] > upper_bound)]

    return outliers 

## Importing data

### Importing data

In [8]:
# read cvs pick_data file
pick_data = pd.read_csv("/Users/ANDRES/Documents/GitHub/OBETAWarehousingAnalyticsProject/Data/003 pick_data.csv", header=None)

/var/folders/0m/9hrzmrmx5rvbnxjrsct8cd500000gn/T/ipykernel_5436/757817874.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  pick_data = pd.read_csv("/Users/ANDRES/Documents/GitHub/OBETAWarehousingAnalyticsProject/Data/003 pick_data.csv", header=None)


Head of data

In [9]:
# print head of data
pick_data.head()

,0,1,2,3,4,5,6,7
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24.0000000
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35.0000000
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50.0000000
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51.0000000
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02.0000000


### Matching PBI Schema

Specify columns names

In [10]:
# rename columns of data to match schema
pick_data.columns = ["product_id", "category", "source_order_id", "order_number", "position_in_order", "pick_volume", "quantity_unit", "date_time"]

Check applied changes

In [11]:
# check head of data to see if columns match
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24.0000000
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35.0000000
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50.0000000
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51.0000000
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02.0000000


Matching data types

In [12]:
# assign data types to pick_data to match schema
pick_data["product_id"] = pick_data["product_id"].astype(str)
pick_data["category"] = pick_data["category"].astype(str)
pick_data["source_order_id"] = pick_data["source_order_id"].astype(str)
pick_data["order_number"] = pick_data["order_number"].astype(str)
pick_data["position_in_order"] = pick_data["position_in_order"].astype(int)
pick_data["pick_volume"] = pick_data["pick_volume"].astype(int)
pick_data["quantity_unit"] = pick_data["quantity_unit"].astype(str)
pick_data["date_time"] = pd.to_datetime(pick_data["date_time"])

In [13]:
# check head of data to evaluate if any change damaged pick_data
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02


In [14]:
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02


### Data description

General information

In [15]:
# check all data types before data manipulation
pick_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33888990 entries, 0 to 33888989
Data columns (total 8 columns):
 #   Column             Dtype         
---  ------             -----         
 0   product_id         object        
 1   category           object        
 2   source_order_id    object        
 3   order_number       object        
 4   position_in_order  int64         
 5   pick_volume        int64         
 6   quantity_unit      object        
 7   date_time          datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 2.0+ GB


## Data manipulation

### Data homogeneity

#### General analysis

General view

In [16]:
# get a general overview
general_overview(pick_data)

The length of the data is:  33888990
-----
-----
product_id
The number of unique product_id is:  97338
-----
category
The number of unique category is:  5
-----
source_order_id
The number of unique source_order_id is:  2
-----
order_number
The number of unique order_number is:  6956136
-----
position_in_order
The number of unique position_in_order is:  340
-----
pick_volume
The min value of pick_volume is:  -2000
The max value of pick_volume is:  200000
-----
quantity_unit
The number of unique quantity_unit is:  9
-----
date_time
The first date_time of the picks_data is:  2011-06-23 00:00:01
The last date_time of the picks_data is:  2020-07-14 11:42:01


#### Specific analysis

##### date_time

Check if **date_time** values are consistent with other columns

In [20]:
# create new column with year of date_time
pick_data["year"] = pick_data.date_time.dt.year
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time,year
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24,2017
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35,2017
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50,2017
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51,2017
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02,2017


In [24]:
# create a loop to evaluation how many products are picked from different categories
year_unique_list = sorted(list(set(pick_data.year.values)))
# year_unique_list = [2015, 2016, 2017, 2018, 2019, 2020]

for i, j in list(combinations(year_unique_list, 2)):
            
    # create a merge dataframe matching the products the categories (warehouses) share with each other
    list_i = list(set(list(pick_data[pick_data["year"]==i]["order_number"].values)))
    list_j = list(set(list(pick_data[pick_data["year"]==j]["order_number"].values)))
    
    # get intersection
    common_values_i_j = set(list_i).intersection(list_j)
    
    # print the merge result
    print(i, " shares ", len(common_values_i_j), " order_number with ", j)

2011  shares  315  order_number with  2012
2011  shares  4  order_number with  2013
2011  shares  20548  order_number with  2014
2011  shares  68285  order_number with  2015
2011  shares  131756  order_number with  2016
2011  shares  21861  order_number with  2017
2011  shares  0  order_number with  2018
2011  shares  59696  order_number with  2019
2011  shares  152358  order_number with  2020
2012  shares  498  order_number with  2013
2012  shares  0  order_number with  2014
2012  shares  284  order_number with  2015
2012  shares  389984  order_number with  2016
2012  shares  48  order_number with  2017
2012  shares  1  order_number with  2018
2012  shares  264  order_number with  2019
2012  shares  165579  order_number with  2020
2013  shares  22  order_number with  2014
2013  shares  0  order_number with  2015
2013  shares  2804  order_number with  2016
2013  shares  286881  order_number with  2017
2013  shares  166  order_number with  2018
2013  shares  0  order_number with  2019
2

* There are **order_number** values that are repeting accross years
* This is bad since we are mixing information from past years with pick orders from later years

Visualize the problem of multiple years in one order

In [ ]:
pick_data[pick_data["order_number"] == "1000574"]

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time,year
1832,303,HRL,48,1000574,1,3,St,2020-05-14 16:26:29,2020
6417080,296320,HRL,48,1000574,1,1,St,2019-05-23 00:04:13,2019
6417254,296324,AKL,48,1000574,2,2,St,2019-05-22 23:35:27,2019
27397895,109910,HRL,48,1000574,6,100,Mt,2015-04-17 04:15:55,2015
27403445,A80713,AKL,48,1000574,5,1,St,2015-04-17 05:48:43,2015


Get the number of registers we should drop if we remove all orders that appear in different years

In [25]:
# create a testing_dataframe for experiments
pick_data_year_drop = pick_data.copy()

# counter variable
drop_registers = 0

# create a loop to evaluation how many products are picked from different categories
year_unique_list = sorted(list(set(pick_data_year_drop.year.values)))

for i, j in list(combinations(year_unique_list, 2)):
            
    # create a merge dataframe matching the products the categories (warehouses) share with each other
    list_i = list(set(list(pick_data_year_drop[pick_data_year_drop["year"]==i]["order_number"].values)))
    list_j = list(set(list(pick_data_year_drop[pick_data_year_drop["year"]==j]["order_number"].values)))
    
    # save intesaction of data
    drop_intersaction_list = [x for x in set(list_i).intersection(list_j)]
    
    # save length of droping registers
    drop_length = len(pick_data_year_drop[pick_data_year_drop["order_number"].isin(drop_intersaction_list)])
    
    # update variable counting
    drop_registers = drop_registers + drop_length
    
    # print progress
    print(i, " shares ", len(common_values_i_j), " order_number with ", j)
    print("Droping: ", drop_length)
    
    # drop registers
    pick_data_year_drop = pick_data_year_drop[~pick_data_year_drop["order_number"].isin(drop_intersaction_list)]
    pick_data_year_drop.reset_index(drop=True, inplace=True)
    
print("Total droped registers: ", drop_registers)

2011  shares  1468  order_number with  2012
Droping:  4207
2011  shares  1468  order_number with  2013
Droping:  69
2011  shares  1468  order_number with  2014
Droping:  185690
2011  shares  1468  order_number with  2015
Droping:  657739
2011  shares  1468  order_number with  2016
Droping:  1250161
2011  shares  1468  order_number with  2017
Droping:  79787
2011  shares  1468  order_number with  2018
Droping:  0
2011  shares  1468  order_number with  2019
Droping:  211131
2011  shares  1468  order_number with  2020
Droping:  553775
2012  shares  1468  order_number with  2013
Droping:  4927
2012  shares  1468  order_number with  2014
Droping:  0
2012  shares  1468  order_number with  2015
Droping:  3827
2012  shares  1468  order_number with  2016
Droping:  3209024
2012  shares  1468  order_number with  2017
Droping:  403
2012  shares  1468  order_number with  2018
Droping:  6
2012  shares  1468  order_number with  2019
Droping:  372
2012  shares  1468  order_number with  2020
Droping:  

In [33]:
len(pick_data_year_drop)

17055506

Second scenario: drop orders in different years after having dropped years before 2017

In [32]:
# create a testing_dataframe for experiments
pick_data_year_drop_2 = pick_data.copy()

# filter year
pick_data_year_drop_2 = pick_data_year_drop_2[pick_data_year_drop_2["year"] >= 2017]
pick_data_year_drop_2.reset_index(drop=True, inplace=True)

# counter variable
drop_registers = 0

# create a loop to evaluation how many products are picked from different categories
year_unique_list = sorted(list(set(pick_data_year_drop_2.year.values)))

for i, j in list(combinations(year_unique_list, 2)):
            
    # create a merge dataframe matching the products the categories (warehouses) share with each other
    list_i = list(set(list(pick_data_year_drop_2[pick_data_year_drop_2["year"]==i]["order_number"].values)))
    list_j = list(set(list(pick_data_year_drop_2[pick_data_year_drop_2["year"]==j]["order_number"].values)))
    
    # save intesaction of data
    drop_intersaction_list = [x for x in set(list_i).intersection(list_j)]
    
    # save length of droping registers
    drop_length = len(pick_data_year_drop_2[pick_data_year_drop_2["order_number"].isin(drop_intersaction_list)])
    
    # update variable counting
    drop_registers = drop_registers + drop_length
    
    # print progress
    print(i, " shares ", len(common_values_i_j), " order_number with ", j)
    print("Droping: ", drop_length)
    
    # drop registers
    pick_data_year_drop_2 = pick_data_year_drop_2[~pick_data_year_drop_2["order_number"].isin(drop_intersaction_list)]
    pick_data_year_drop_2.reset_index(drop=True, inplace=True)
    
print("Total droped registers: ", drop_registers)

2017  shares  1468  order_number with  2018
Droping:  38
2017  shares  1468  order_number with  2019
Droping:  80
2017  shares  1468  order_number with  2020
Droping:  0
2018  shares  1468  order_number with  2019
Droping:  41
2018  shares  1468  order_number with  2020
Droping:  10
2019  shares  1468  order_number with  2020
Droping:  12581
Total droped registers:  12750


In [34]:
len(pick_data_year_drop_2)

16046699

##### product_id

Check if **product_id** values are homogeneous

In [17]:
pick_data.head()

,product_id,category,source_order_id,order_number,position_in_order,pick_volume,quantity_unit,date_time
0,2,SHL,48,7055448,1,29,St,2017-06-30 11:15:24
1,2,SHL,48,7055448,1,30,St,2017-06-30 11:22:35
2,2,SHL,48,7055448,1,30,St,2017-06-30 12:04:50
3,2,SHL,48,7055448,1,20,St,2017-06-30 12:04:51
4,2,SHL,48,7055448,1,30,St,2017-06-30 12:05:02


In [18]:
all(x.isdigit() for x in list(set(list(pick_data["product_id"].values))))

False

* **product_id** values are not always numerical

##### category

Check if products come from the same category everytime

In [ ]:
# check data in category
print(set(pick_data.category.values))

In [ ]:
# create a loop to evaluation how many products are picked from different categories

category_unique_list = list(set(pick_data.category.values))

for i, j in list(combinations(category_unique_list, 2)):
            
            # create a merge dataframe matching the products the categories (warehouses) share with each other
            list_i = list(set(list(pick_data[pick_data["category"]==i]["product_id"].values)))
            list_j = list(set(list(pick_data[pick_data["category"]==j]["product_id"].values)))
            
            # compare lists to identify common values
            common_values_i_j = [x for x in list_i if x in list_j]
            
            # print the merge result
            print(i, " shares ", len(common_values_i_j), " product_id with ", j)

* Same **product_id** values can be picked from **different category** values

In [ ]:
pick_data.pivot_table(
    index="category",
    columns="source_order_id",
    values="position_in_order",
    aggfunc="max")

> When the **source_order_id** is **46**, the **max position_in_order** is no grater than **85** in any warehouse

> Warehouses **AKL**, **Manuell**, **SHL** are the houses with greater **position_in_order**

In [ ]:
pick_data.pivot_table(
    index="category",
    columns="source_order_id",
    values="order_number",
    aggfunc="count")

> Warehouses **AKL**, **HRL**, **SHL** have the **greater number of orders received** accross order types (**46**, **48**)

##### source_order_id

Check if **source_order_id** can vary in the same **order_number**

In [ ]:
# check data in source_order_id
print(set(pick_data.source_order_id.values))

In [ ]:
pick_data.head()

In [ ]:
# check how many times 48 and 46 are sharing orders accross the entire dataset
order_source_set_48 = list(set(list(pick_data[pick_data["source_order_id"] == "48"]["order_number"].values)))
order_source_set_46 = list(set(list(pick_data[pick_data["source_order_id"] == "46"]["order_number"].values)))

# common orders between source_order_id
# common_source_order_order_number = [x for x in order_source_set_48 if x in order_source_set_46]
common_source_order_order_number = set(order_source_set_48).intersection(order_source_set_46)

print("The number of same order_number shared between 46 & 48: ", len(common_source_order_order_number))

In [ ]:
pick_data.info()

In [ ]:
pick_data[pick_data["source_order_id" == "48"]]

In [ ]:
# check how many times 48 and 46 are sharing orders accross the entire dataset
order_source_set_48 = list(set(list(pick_data[(pick_data["source_order_id"] == "48") & (pick_data["year"] == 2013)]["order_number"].values)))
order_source_set_46 = list(set(list(pick_data[(pick_data["source_order_id"] == "46") & (pick_data["year"] == 2013)]["order_number"].values)))

# common orders between source_order_id
# common_source_order_order_number = [x for x in order_source_set_48 if x in order_source_set_46]
common_source_order_order_number = set(order_source_set_48).intersection(order_source_set_46)

print("The number of same order_number shared between 46 & 48: ", len(common_source_order_order_number))

##### quantity_unit

Check if quantity units are homogeneous for a **product_id**

In [ ]:
# check data in source_order_id
print(set(pick_data.quantity_unit.values))

In [ ]:
# create empty dict for interation
quantity_unit_product_dict = {}

# create a set of unique product_id values for each quantity unit
product_list_St = list(set(pick_data[pick_data["quantity_unit"] == "St"]["product_id"].values))
product_list_Mt = list(set(pick_data[pick_data["quantity_unit"] == "Mt"]["product_id"].values))
product_list_Ei = list(set(pick_data[pick_data["quantity_unit"] == "Ei"]["product_id"].values))
product_list_kg = list(set(pick_data[pick_data["quantity_unit"] == "kg"]["product_id"].values))
product_list_Ro = list(set(pick_data[pick_data["quantity_unit"] == "Ro"]["product_id"].values))
product_list_Bl = list(set(pick_data[pick_data["quantity_unit"] == "Bl"]["product_id"].values))
product_list_VE = list(set(pick_data[pick_data["quantity_unit"] == "VE"]["product_id"].values))
product_list_Pa = list(set(pick_data[pick_data["quantity_unit"] == "Pa"]["product_id"].values))
product_list_Se = list(set(pick_data[pick_data["quantity_unit"] == "Se"]["product_id"].values))

# configure dictionary
quantity_unit_product_dict["product_list_St"] = product_list_St
quantity_unit_product_dict["product_list_Mt"] = product_list_Mt
quantity_unit_product_dict["product_list_Ei"] = product_list_Ei
quantity_unit_product_dict["product_list_kg"] = product_list_kg
quantity_unit_product_dict["product_list_Ro"] = product_list_Ro
quantity_unit_product_dict["product_list_Bl"] = product_list_Bl
quantity_unit_product_dict["product_list_VE"] = product_list_VE
quantity_unit_product_dict["product_list_Pa"] = product_list_Pa
quantity_unit_product_dict["product_list_Se"] = product_list_Se

# common orders between source_order_id
for i, j in list(combinations(list(quantity_unit_product_dict.keys()), 2)):
    common_product_id_quantity_unit = set(quantity_unit_product_dict[i]).intersection(quantity_unit_product_dict[j])
    print("The quantity_unit ", i[-2:], " shares ", len(common_product_id_quantity_unit), " product_id with ", j[-2:])

In [ ]:
set(quantity_unit_product_dict["product_list_St"]).intersection(quantity_unit_product_dict["product_list_Mt"])

In [ ]:
# len(pick_data[pick_data["product_id"] == '451123'])
pick_data[(pick_data["product_id"] == '451123') & (pick_data["quantity_unit"] == 'St')][0:1]

In [ ]:
# len(pick_data[pick_data["product_id"] == '451123'])
pick_data[(pick_data["product_id"] == '451123') & (pick_data["quantity_unit"] == 'Mt')][0:1]

* We see that there are some **product_id** values that are measured in **different quantities**

##### position_in_order

In [ ]:
# check data in position_in_order
position_in_order_set_list = list(set([int(x) for x in pick_data.position_in_order.values]))
print(position_in_order_set_list)

* **position_in_order** seems to be **semi-continous** so it's a good idea to check some examples of how that variable behaviors in some **order_number** cases

Create a scatter plot over variable **position_in_order** to evaluate variable behavior

In [ ]:
plt.scatter(position_in_order_set_list, position_in_order_set_list)
plt.title("Line Plot to Evaluate Continuity - position_in_order")
plt.xlabel("position_in_order")
plt.ylabel("position_in_order_set_list")
plt.grid(True)
plt.show()

* We clearly see that there's a break in the sequence after max position > 300

Check number of breaks in the sequence of **position_in_order**

In [ ]:
sequential_count = count_non_sequential(position_in_order_set_list)
print("Max. position: ", sorted(position_in_order_set_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(position_in_order_set_list) == 0:
    print("The position_in_order_set_list: IS sequential")
else:
    print("The position_in_order_set_list: IS NOT sequential")

* **position_in_order** is a **non-continuous semi-continuous** variable with **two breaks**

> We proceed to theck the behavior of the variable **position_in_order** within a giving **order_number**

Check if **position_in_order** is a continues sequence from **1** to **n**

In [ ]:
# check random order_number with position_in_order > 10
random_position = pick_data[pick_data["position_in_order"] > 10].head(1)["order_number"].values[0]
# pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")
pick_data[pick_data["order_number"] == random_position].sort_values(by="date_time")

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

In [ ]:
# check random order_number with position_in_order > 100
random_position = pick_data[pick_data["position_in_order"] > 100].head(1)["order_number"].values[0]
pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

In [ ]:
# check random order_number with position_in_order > 200
random_position = pick_data[pick_data["position_in_order"] > 200].head(1)["order_number"].values[0]
pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

In [ ]:
# check random order_number with position_in_order > 250
random_position = pick_data[pick_data["position_in_order"] > 250].head(1)["order_number"].values[0]
pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

In [ ]:
# check random order_number with position_in_order > 300
random_position = pick_data[pick_data["position_in_order"] > 300].head(1)["order_number"].values[0]
pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

In [ ]:
# check random order_number with position_in_order > 400
random_position = pick_data[pick_data["position_in_order"] > 400].head(1)["order_number"].values[0]
pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")
# pick_data[(pick_data["order_number"] == random_position) & (pick_data["category"] == "SHL")].sort_values(by="date_time")

In [ ]:
len(pick_data[pick_data["year"] == 2013])

In [ ]:
# check if values in position_in_order are sequential
sorted_list = [int(x) for x in set(pick_data[pick_data["order_number"] == random_position].sort_values(by="position_in_order")["position_in_order"].values)]
sequential_count = count_non_sequential(sorted_list)
print("Max. position: ", sorted(sorted_list)[-1])
print("Number of breaks: ", sequential_count)
if count_non_sequential(sorted_list) == 0:
    print("The order_number: ", random_position, " IS sequential")
else:
    print("The order_number: ", random_position, " IS NOT sequential")

> From the previous analysis we can check different things:

1) The grater the max position_in_order, the more number of breaks in the the sequence of the   variable (1; n)

2) We have to evaluate what to do with this order_number, since they could damage future analysis because of the breakes in the sequence

### Missing values

Checking missing values over all columns

In [ ]:
# check count of missing values for each variable 
pick_data.isna().sum()

* Since there are no missing values, we can assume that the data is complete and we can proceed to validate the consistency of the variables

### Duplicates

#### General analysis

Check duplicates based on **all columns**

In [ ]:
# get duplicates dataframe over the all columns
duplicates_pick_date = pick_data.duplicated(subset = list(pick_data.columns))
pick_data[duplicates_pick_date].head()

* We see that there are perfect duplicates

Get total length of perfect matches

In [ ]:
# get the count of duplicated values
print(len(pick_data[duplicates_pick_date]))

* We see that **8.024** rows are duplicates evaluating **all columns**

In [ ]:
pick_data[duplicates_pick_date].pivot_table(
    index="category",
    columns="source_order_id",
    values="order_number",
    aggfunc="count")

> Warehouse **SHL**, **Manuell**, **Kabellager** have the most number of perfect duplicates

> Warehouses **AKL** and **HRL** are the ones with less duplicates

#### Specific analysis

Check one duplicate example based on **all** column before taking any decission

In [ ]:
# check the duplicated cases for row 14 over all columns (because we saw the index in the previous output)
index_to_check = 1327
columns_to_evaluate = list(pick_data.columns)

check_duplicates_row_columns(pick_data, index_to_check, columns_to_evaluate)

* This is one of the **8.024** cases where registers are **exactly the same**

> Comparing this register with another register that doesn't follow this behavior

Getting registers **outside** the behavior of perfect duplicates

In [ ]:
pick_data[~duplicates_pick_date].head()

Check one **orde_number** that is **outside** the behavior of perfect duplicates

In [ ]:
pick_data[pick_data["order_number"]=="7055448"].head()

* We see the **picks** are placed in a way that **date_time** and **pick_volume** could actually **overlap** within an **order_number**

> **Not recommendable to drop duplicates for several reasons:**

    1) We don't have a unique ID for pick_data, wich makes it difficult to make decissions over duplicates

    2) Orders are placed in a way that date_time and pick_volme can overlap within an order_number

### Outliers

#### Idenfy outliers

> For outliers identification we need to understand the variable where they could be allocated

Overview of pick_data 

In [ ]:
# checking structure of data
pick_data.info()

In [ ]:
# get a general overview
general_overview(pick_data)

In [ ]:
pick_data.head()

* We are going to execute the outlier identification over all **integer** variables:

    1) **pick_volume**
    2) **position_in_order**

> **pick_volume** has to be evaluated by product

> **position_in_order** was already evaluated in *data homogeneity*

###### pick_volume

> Remember that the **pick_volume** from a **single product_id** can be measured with **different quantity_unit**

Check value ranges over **pick_volume** the dataframe

In [ ]:
print(pick_data["pick_volume"].min())
print(pick_data["pick_volume"].max())

* The range in **pick_volumne** has to greater than 0 

Check the dataframe filtered by **undesired pick_volume range**

In [ ]:
pick_data[pick_data["pick_volume"]<=0]

* There are **190.371 registers** that are **adding noise** to the dataframe

Identify the **warehoses** responsable from this **product_id** invalid **pick_volume** values

In [ ]:
pick_data[pick_data["pick_volume"]==0].pivot_table(
    index=["category", "year"],
    columns="source_order_id",
    values="product_id",
    aggfunc="count")

In [ ]:
pick_data[pick_data["pick_volume"]<0].pivot_table(
    index=["category", "year"],
    columns="source_order_id",
    values="product_id",
    aggfunc="count")

In [ ]:
len(pick_data[pick_data["pick_volume"]<0])

> Most of the **zero** pick_volume are alocated in AKL

> All the **negative pick_volume** are alocated in **SHL**

In [ ]:
pick_data.info()

Drop 190.371 registers (see above)

In [ ]:
pick_data = pick_data[pick_data["pick_volume"]>0].reset_index(drop=True)
pick_data.head()

In [ ]:
# get a general overview
general_overview(pick_data)

Boxplot to evaluate frequency of pick_volume over **all data**

In [ ]:
pick_volume_boxplot = pick_data.boxplot(column=['pick_volume'], vert = False)

Check highest **pick_volume** values

In [ ]:
pick_data[pick_data["pick_volume"] > 25000]

Check number of outliers from a **product_id**

In [50]:
product_id = "500103"

In [ ]:
product_id_volume_outliers(0.05, 0.95, pick_data, product_id)

In [ ]:
pick_data[pick_data["product_id"] == product_id].boxplot(column=['pick_volume'], vert = False)
plt.show()

Check order

In [ ]:
pick_data[pick_data["order_number"] == "7981876"]

## Upload

Write csv file

In [55]:
pick_data.to_csv("pick_data_upload.csv", index=False)

Check that csv file is well written

In [ ]:
pick_dataest = pd.read_csv("pick_data_upload.csv")
pick_dataest.head()

## Paralel Work

### Nacho

I need from Python a **csv** file with only the **order_number** column.

Basically I need to:

    * Take this column from the DataFrame
    * Remove duplicates
    * Sort the values
    * Same the file with the name "picks_orders".


In [67]:
# # create set list of unique values of column order_number
# order_number_set_list = list(set(list(pick_data["order_number"].values)))
# order_number_set_list = sorted(order_number_set_list)

# # convert the list into a data frame
# df_order_number_set_list = pd.DataFrame(order_number_set_list, columns=["order_number"])

# # define the csv file name
# filename = "picks_orders.csv"

# # write the data frame to a csv file
# df_order_number_set_list.to_csv(filename, index=False)